In [6]:
import pickle
from models import String, Gamma, Air, SoundBoard
import numpy as np
import scipy.sparse as sp
import cupy as cp
import cupyx
import cupyx.scipy.sparse.linalg
from tqdm import tqdm


In [3]:
constants: dict[str, sp.spmatrix | cp.sparse.spmatrix]
string: String
soundboard: SoundBoard
air: Air
gamma: Gamma
constants, (string, soundboard, air, gamma) = pickle.load(open('constants.pkl', 'rb'))

for key, value in constants.items():
    if isinstance(constants[key], sp.spmatrix):
        constants[key] = cupyx.scipy.sparse.csr_matrix(value)


In [3]:
print(f"{constants['M_sh'].shape = }")
print(f"{constants['M_qh'].shape = }")
print(f"{constants['M_Mh'].shape = }")
print(f"{constants['M_ph'].shape = }")
print(f"{constants['M_ah'].shape = }")
print(f"{constants['M_pah'].shape = }")
print(f"{constants['D_h'].shape = }")
print(f"{constants['J_h'].shape = }")
print(f"{constants['H_h'].shape = }")
print(f"{constants['B_w_h'].shape = }")
print(f"{constants['Gh'].shape = }")


constants['M_sh'].shape = (99, 99)
constants['M_qh'].shape = (100, 100)
constants['M_Mh'].shape = (61200, 61200)
constants['M_ph'].shape = (20400, 20400)
constants['M_ah'].shape = (382500, 382500)
constants['M_pah'].shape = (125000, 125000)
constants['D_h'].shape = (99, 100)
constants['J_h'].shape = (20400, 100)
constants['H_h'].shape = (61200, 20400)
constants['B_w_h'].shape = (44761, 20400)
constants['Gh'].shape = (382500, 125000)


In [4]:
def inv(A: cp.sparse.spmatrix, batch_size: int = 1000) -> cp.sparse.spmatrix:
    assert A.shape[0] == A.shape[1]
    assert batch_size > 0

    n = A.shape[0]

    columns = cp.empty((0,), dtype=cp.int32)
    rows = cp.empty((0,), dtype=cp.int32)
    values = cp.empty((0,), dtype=A.dtype)

    A_factorized = cupyx.scipy.sparse.linalg.factorized(A)
    for i in tqdm(range(0, n, batch_size)):
        b = cp.zeros((n, batch_size))
        for j in range(min(batch_size, n-i)):
            b[i+j, j] = 1
        x = A_factorized(b)
        
        nonzeros = cp.asnumpy(x).nonzero()
        nonzeros_gpu = cp.asarray(nonzeros)
        columns = cp.concatenate((columns, nonzeros_gpu[0]), axis=0)
        rows = cp.concatenate((rows, nonzeros_gpu[1] + i), axis=0)
        values = cp.concatenate((values, x[nonzeros]), axis=0)

    return cupyx.scipy.sparse.coo_matrix((values, (rows, columns)), shape=(n, n))

def set_tol(A: cp.sparse.coo_matrix, tol: float):
    A.data[cp.abs(A.data) < tol] = 0
    A.eliminate_zeros()


In [4]:
for key in list(constants.keys()):
    if key.startswith('M') and not key.endswith('inverse'):
        if key + '_inverse' not in constants:
            print(key)

In [18]:
cp.get_default_memory_pool().free_all_blocks()

In [21]:
def gpu_to_cpu():
    for k, v in constants.items():
        if isinstance(v, cp.sparse.spmatrix):
            constants[k] = sp.csr_matrix(v.get())
    cp.get_default_memory_pool().free_all_blocks()

def cpu_to_gpu():
    for k, v in constants.items():
        if isinstance(v, sp.spmatrix):
            constants[k] = cp.sparse.csr_matrix(v)

In [ ]:
for key in list(constants.keys()):
    if key.startswith('M') and not key.endswith('inverse'):
        if key + '_inverse' not in constants:
            # calculate the inverse
            print(f"Calculating {key}")
            constants[key + '_inverse'] = inv(cp.sparse.coo_matrix(constants[key]))
        else:
            # check if the inverse is correct in another device
            print(f"Checking {key}")
            I = constants[key + '_inverse'] * constants[key]
            set_tol(I, 1e-10)
            assert I.nnz == I.shape[0]
            assert cp.allclose(I.diagonal(), 1)
    

In [ ]:
pickle.dump((constants, (string, soundboard, air, gamma)), open('constants.pkl', 'wb'))

In [120]:
cpu_to_gpu()

In [18]:
len(omega_n)

50

In [84]:
constants['dt'] = 0.1
variables = {
    # variables on the string
    'v_s': np.zeros((constants['Ns']-1)),    # v_s is piecewise constant on the string
    'q': np.zeros((constants['Ns'])),         # q_s is piecewise linear on the string

    # variables on the soundboard
    'v_p': np.zeros((soundboard.num_vertices + soundboard.num_edges + soundboard.num_centers)),
    'M_p': np.zeros((soundboard.num_vertices + soundboard.num_edges + soundboard.num_centers) * 3),

    # variables in the air
    'v_a': np.zeros(3 * (air.resolution[0] * air.resolution[1] * air.resolution[2]) + air.resolution[0] * air.resolution[1] + air.resolution[1] * air.resolution[2] + air.resolution[0] * air.resolution[2]),
    'p_a': np.zeros(air.resolution[0] * air.resolution[1] * air.resolution[2]),
    
    # variables on the gittar body
    'lambda': cp.zeros((len(gamma.vertices))),
}

for k, v in variables.items():
    print(k, v.shape, sep='\t')



v_s	(99,)
q	(100,)
v_p	(20400,)
M_p	(61200,)
v_a	(382500,)
p_a	(125000,)
lambda	(44761,)


In [124]:
constants['K_h'].shape

(20400, 20400)

In [127]:
denseK_h = constants['K_h'].todense()

array([[ 6.06316890e-06, -2.36713393e-06,  2.03974070e-06, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.15629451e-06,  5.19527483e-06, -2.62461209e-06, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.80820770e-06, -2.74120046e-06,  5.08111544e-06, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         7.79486611e-07, -6.55819568e-10, -1.23879834e-08],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.25677576e-10,  7.61542531e-07, -1.90321641e-08],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.86651554e-08,  1.31322886e-08,  7.04790621e-07]])

array([             nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan, -4.58717617e+241,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,

In [170]:
constants['n_specturm'] = 3
constants['K_h'] = constants['H_h'].T * constants['M_Mh'] * constants['H_h']
constants['omega_n'] = cupyx.scipy.sparse.linalg.eigsh(constants['K_h'], k=constants['n_specturm'], which='LA', return_eigenvectors=False)
constants['omega_n']

array([8.07541306e-05, 8.13910275e-05, 8.24117595e-05])

In [61]:
gpu_to_cpu()

            len(vp)         len(va)     len(lambda)     len(q)

len(vp)         1            0          c B_omega        c J

len(va)         0           M_h/dt      -B_gamma^T       0

len(lambda)     2B_omega    -B_gamma        0            0

len(q)         -J^T        0               0           Mq/dt

            20400       382500      44761        100
20400

382500

44761

100         

In [111]:
lengths = [
    len(variables['v_p']),
    len(variables['v_a']),
    len(variables['lambda']),
    len(variables['q']),
]

lengths

[20400, 382500, 44761, 100]

In [270]:
def cos_sqrt(M: cp.sparse.spmatrix, n_specturm: int):
    # make sure M is positive definite
    # if M is positive definite
    # if M = V Lamda V^T

    eigvals, eigvects  = cupyx.scipy.sparse.linalg.eigsh(M, k=n_specturm, which='SA', return_eigenvectors=True, tol=1e-10)
    for i in range(n_specturm):
        print(eigvals[i])
        print(eigvects[:, i])
    
    return eigvects @  cp.diag(cp.cos(cp.sqrt(eigvals))) @ eigvects.T

M = cp.sparse.csc_matrix(
    (cp.array([1, 2, 3, 3, 2, 6.]),
    (cp.array([0, 1, 2, 1, 2, 3]), 
     cp.array([0, 1, 2, 2, 1, 3]))),
    shape=(5, 5)
)
# print(M.todense())


# calculate the eigenvalues
eigenvalues = cp.linalg.eigvalsh(M.todense())
eigenvalues

cos_sqrt(M, 2)
pass

-0.015564419581502401
[-0.03789123  0.20127893 -0.19165861 -0.00556507  0.95983699]
1.0482667897616447
[0.99916072 0.01854816 0.00368782 0.00159973 0.03629968]


In [215]:
eig_result = np.linalg.eig(M.get().todense())
print(eig_result.eigenvalues)
eig_result.eigenvectors

[5. 0. 1. 6. 0.]


matrix([[ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.70710678,  0.83205029,  0.        ,  0.        ,  0.        ],
        [ 0.70710678, -0.5547002 ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ]])

In [246]:
(eig_result.eigenvectors)[:, :2] @ np.diag(eig_result.eigenvalues)[:2,:2] @ np.linalg.inv(eig_result.eigenvectors)[:2]

matrix([[0., 0., 0., 0., 0.],
        [0., 2., 3., 0., 0.],
        [0., 2., 3., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [178]:
gpu_to_cpu()


all_blocks = [[sp.csr_matrix((lengths[i], lengths[j])) for j in range(4)] for i in range(4)]


all_blocks[0][0] = sp.identity(len(variables['v_p']))

all_blocks[1][1] = constants['M_ah'] / constants['delta_t']

# all_blocks[2][0] = 2 * constants['B_omega_h']

all_blocks[2][1] = 2 * -constants['B_gamma_h']
all_blocks[1][2] = 2 * -constants['B_gamma_h'].T

all_blocks[3][0] = -constants['J_h'].T
all_blocks[3][3] = constants['M_qh'].T



import scipy
for omega in constants['omega_n']:
    print(omega)
    # K = omage
    C = 1 - scipy.cos
    all_blocks[0][2] = 1-cos()
# all_blocks[0][3] = 


all_blocks = sp.bmat(all_blocks)

all_blocks

8.075413063733712e-05
8.13910274957755e-05
8.241175952704385e-05


<447761x447761 sparse matrix of type '<class 'numpy.float64'>'
	with 1852399 stored elements in COOrdinate format>

In [106]:
constants['B_gamma_h'].T.shape

(382500, 44761)

In [107]:
constants['J_h']

<20400x100 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [ ]:
,

In [93]:
constants['D_h'].T.shape

(100, 99)

In [90]:
constants['M_qh'].shape

(100, 100)

In [86]:
# left

(constants['J_h'].T * variables['v_p'] + constants['M_qh'] * variables['q']) 

(100,)

In [87]:
constants['M_ah']

<382500x382500 sparse matrix of type '<class 'numpy.float64'>'
	with 1132500 stored elements in Compressed Sparse Row format>

In [88]:
variables['v_a'].shape

(382500,)

In [ ]:
# right
constants['M_qh'] * variables['v_s'] + 

In [79]:
constants['J_h'].T

<100x20400 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Column format>

In [65]:
sp.hstack([constants['J_h'].T, sp.csr_matrix((3, 3)), 0, constants['M_qh']])

ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 1, expected 100.

In [75]:
constants['J_h'].T.shape

(100, 20400)

In [35]:
J_h_T = constants['J_h'].T  # (100, 20400)
M_a_h = constants['M_ah']   # (382500, 382500)
B_T_h = sp.csr_matrix([[1]])  
B_omega_h = sp.csr_matrix([[2]])  
K_h = sp.csr_matrix([[1]])   
M_q_h = sp.csr_matrix([[1]]) 
dt = 0.1

# 计算一些中间值
val_1 = sp.val_2 = B_T_h / dt
val_3 = J_h_T / (2 * dt)

# 拼接大矩阵
row1 = sp.hstack([-J_h_T, sp.csr_matrix((1, 1)), sp.csr_matrix((1, 1)), sp.csr_matrix((1, 1)), M_q_h / dt])
row2 = sp.hstack([sp.csr_matrix((1, 1)), M_a_h / dt, sp.csr_matrix((1, 1)), -B_T_h, sp.csr_matrix((1, 1))])
row3 = sp.hstack([2 * B_omega_h, -B_T_h, sp.csr_matrix((1, 1)), sp.csr_matrix((1, 1)), sp.csr_matrix((1, 1))])
row4 = sp.hstack([sp.csr_matrix((1, 1)), sp.csr_matrix((1, 1)), val_1, val_2, val_1])
row5 = sp.hstack([sp.csr_matrix((1, 1)), sp.csr_matrix((1, 1)), sp.csr_matrix((1, 1)), sp.csr_matrix((1, 1)), val_3])

big_matrix = sp.vstack([row1, row2, row3, row4, row5])

print(big_matrix.toarray())  # 打印稠密矩阵以验证


TypeError: expected dimension <= 2 array or matrix

In [ ]:

import numpy as np
import scipy.sparse as sp

class PhysicalSimulation:
    def __init__(self, constants, soundboard, air, gamma):
        self.constants = constants
        self.time_step = constants['dt']
        
        self.variables = {
            'v_s': sp.csr_matrix((constants['Ns']-1, 1)),  # Piecewise constant on the string
            'q': sp.csr_matrix((constants['Ns'], 1)),       # Piecewise linear on the string

            # Variables on the soundboard
            'v_p': sp.csr_matrix((soundboard.num_vertices + soundboard.num_edges + soundboard.num_centers, 1)),
            'M_p': sp.csr_matrix(((soundboard.num_vertices + soundboard.num_edges + soundboard.num_centers) * 3, 1)),

            # Variables in the air
            'v_a': sp.csr_matrix((3 * (air.resolution[0] * air.resolution[1] * air.resolution[2]) + 
                                  air.resolution[0] * air.resolution[1] + 
                                  air.resolution[1] * air.resolution[2] + 
                                  air.resolution[0] * air.resolution[2], 1)),
            'p_a': sp.csr_matrix((air.resolution[0] * air.resolution[1] * air.resolution[2], 1)),

            # Variables on the guitar body
            'lambda': sp.csr_matrix((len(gamma.vertices), 1))
        }
        
        
        for k, v in variables.items():
            print(k, v.shape, sep='\t')

        # Initialize history for each variable
        self.history = {k: [v.copy()] for k, v in self.variables.items()}

        self.constants['K_h'] = self.constants['H_h'].T * self.constants['M_Mh'] * self.constants['H_h']
        self.constants['omega_n'] = cupyx.scipy.sparse.linalg.eigsh(self.constants['K_h'], k=self.constants['n_specturm'], which='SA')
        

    def update(self):
        # Placeholder for the actual update logic
        for key, value in self.variables.items():
            # This is where the update equations would go
            # For now, we'll just add a small random increment for demonstration purposes
            self.variables[key] += sp.csr_matrix(np.random.randn(*value.shape) * self.time_step)

            # Store the updated state in history
            self.history[key].append(self.variables[key].copy())

    def get_history(self):
        return self.history

# Example usage:

class Soundboard:
    num_vertices = 100
    num_edges = 50
    num_centers = 20

class Air:
    resolution = (10, 10, 10)

class Gamma:
    vertices = range(30)

constants = {'dt': 0.1, 'Ns': 50}
soundboard = Soundboard()
air = Air()
gamma = Gamma()

simulation = PhysicalSimulation(constants, soundboard, air, gamma)

# Update the simulation for 10 time steps
for _ in range(10):
    simulation.update()

history = simulation.get_history()

# Print history shapes to verify
for k, v in history.items():
    print(f"{k}: {len(v)} time steps, shape {v[0].shape}")


class Simulation:

    history: list

    def __init__(self, variables, constants):
        self.variables = variables
        self.constants = constants
        

    def step(self):


In [20]:
cp.cuda.Device(1).use()

<CUDA Device 1>

In [26]:
cpu_to_gpu()

In [25]:
gpu_to_cpu()

In [31]:
constants['delta_t'] = 1 / 50000